In [1]:
import pandas as pd

In [2]:
df = pd.read_pickle('data.pkl')

In [3]:
df = df[[
    'date_block_num',
    'shop_id',
    'item_id',
    'item_cnt_month',
    'zip_code',
    'item_category_id',
    'category_type',
    'category_device',
    'item_cnt_month_lag_1',
    'item_cnt_month_lag_2',
    'item_cnt_month_lag_3',
    'item_cnt_month_lag_6',
    'item_cnt_month_lag_12',
    'date_avg_item_cnt_lag_1',
    'date_item_avg_item_cnt_lag_1',
    'date_item_avg_item_cnt_lag_2',
    'date_item_avg_item_cnt_lag_3',
    'date_item_avg_item_cnt_lag_6',
    'date_item_avg_item_cnt_lag_12',
    'date_shop_avg_item_cnt_lag_1',
    'date_shop_avg_item_cnt_lag_2',
    'date_shop_avg_item_cnt_lag_3',
    'date_shop_avg_item_cnt_lag_6',
    'date_shop_avg_item_cnt_lag_12',
    'date_cat_avg_item_cnt_lag_1',
    'date_shop_cat_avg_item_cnt_lag_1',
    'date_city_avg_item_cnt_lag_1',
    'date_item_city_avg_item_cnt_lag_1',
    'delta_price_trend_with_lag_1',
    'month',
    'days_in_month',
    'item_last_sold_in_given_shop',
    'item_last_sold',
    'item_first_sold_in_given_shop',
    'item_first_sold'
]]

In [4]:
df_og = pd.read_pickle('data_og.pkl')

df_og = df_og[[
    'date_block_num',
    'shop_id',
    'item_id',
    'item_cnt_month',
    'city_code',
    'item_category_id',
    'type_code',
    'subtype_code',
    'item_cnt_month_lag_1',
    'item_cnt_month_lag_2',
    'item_cnt_month_lag_3',
    'item_cnt_month_lag_6',
    'item_cnt_month_lag_12',
    'date_avg_item_cnt_lag_1',
    'date_item_avg_item_cnt_lag_1',
    'date_item_avg_item_cnt_lag_2',
    'date_item_avg_item_cnt_lag_3',
    'date_item_avg_item_cnt_lag_6',
    'date_item_avg_item_cnt_lag_12',
    'date_shop_avg_item_cnt_lag_1',
    'date_shop_avg_item_cnt_lag_2',
    'date_shop_avg_item_cnt_lag_3',
    'date_shop_avg_item_cnt_lag_6',
    'date_shop_avg_item_cnt_lag_12',
    'date_cat_avg_item_cnt_lag_1',
    'date_shop_cat_avg_item_cnt_lag_1',
    'date_city_avg_item_cnt_lag_1',
    'date_item_city_avg_item_cnt_lag_1',
    'delta_price_lag',
    'month',
    'days',
    'item_shop_last_sale',
    'item_last_sale',
    'item_shop_first_sale',
    'item_first_sale',
]]

In [5]:
df = df.reset_index(drop=True)
df_og = df_og.reset_index(drop=True)

In [6]:
df['delta_price_trend_with_lag_1'] = df_og['delta_price_lag']
df['item_last_sold_in_given_shop'] = df_og['item_shop_last_sale']
df['item_last_sold'] = df_og['item_last_sale']
df['item_first_sold_in_given_shop'] = df_og['item_shop_first_sale']
df['item_first_sold'] = df_og['item_first_sale']

df['zip_code'] = df_og['city_code']
df['category_type'] = df_og['type_code']
df['category_device'] = df_og['subtype_code']

In [7]:
X_train = df[df.date_block_num < 33].drop(['item_cnt_month'], axis=1)
Y_train = df[df.date_block_num < 33]['item_cnt_month']
X_valid = df[df.date_block_num == 33].drop(['item_cnt_month'], axis=1)
Y_valid = df[df.date_block_num == 33]['item_cnt_month']
X_test = df[df.date_block_num == 34].drop(['item_cnt_month'], axis=1)

In [8]:
# df = df[df.date_block_num == 20]
# df = df[df.shop_id == 2]
#
# df_og = df_og[df_og.date_block_num == 20]
# df_og = df_og[df_og.shop_id == 2]
#
# df.to_csv('test.csv')
# df_og.to_csv('test2.csv')

In [9]:
from xgboost import XGBRegressor

model = XGBRegressor(
    max_depth=8,
    n_estimators=1000,
    min_child_weight=300,
    colsample_bytree=0.8,
    subsample=0.8,
    eta=0.3,
    seed=42)

model.fit(
    X_train,
    Y_train,
    eval_metric="rmse",
    eval_set=[(X_train, Y_train), (X_valid, Y_valid)],
    verbose=True,
    early_stopping_rounds = 10)

test = pd.read_csv('data/technical/test.csv')

Y_pred = model.predict(X_valid).clip(0, 20)
Y_test = model.predict(X_test).clip(0, 20)

submission = pd.DataFrame({
    "ID": test.index,
    "item_cnt_month": Y_test
})
submission.to_csv('xgb_submission.csv', index=False)

[0]	validation_0-rmse:1.06052	validation_1-rmse:1.04648
[1]	validation_0-rmse:0.96791	validation_1-rmse:0.98123
[2]	validation_0-rmse:0.91880	validation_1-rmse:0.95063
[3]	validation_0-rmse:0.88849	validation_1-rmse:0.93218
[4]	validation_0-rmse:0.86905	validation_1-rmse:0.92219
[5]	validation_0-rmse:0.85801	validation_1-rmse:0.91719
[6]	validation_0-rmse:0.84914	validation_1-rmse:0.91316
[7]	validation_0-rmse:0.84108	validation_1-rmse:0.91167
[8]	validation_0-rmse:0.83713	validation_1-rmse:0.91149
[9]	validation_0-rmse:0.83285	validation_1-rmse:0.91258
[10]	validation_0-rmse:0.82951	validation_1-rmse:0.91114
[11]	validation_0-rmse:0.82722	validation_1-rmse:0.91347
[12]	validation_0-rmse:0.82503	validation_1-rmse:0.91275
[13]	validation_0-rmse:0.82288	validation_1-rmse:0.91477
[14]	validation_0-rmse:0.82183	validation_1-rmse:0.91468
[15]	validation_0-rmse:0.82072	validation_1-rmse:0.91480
[16]	validation_0-rmse:0.81936	validation_1-rmse:0.91402
[17]	validation_0-rmse:0.81844	validation